### TRAIN.CSV

In [7]:
import py7zr
from pathlib import Path

raw_dir = Path("data/raw")

for file in raw_dir.glob("*.7z"):
    with py7zr.SevenZipFile(file, mode='r') as archive:
        archive.extractall(path=raw_dir)


### 1) Load and inspect shapes/types and missing values:

In [ ]:

import pandas as pd, numpy as np
from pathlib import Path

PATH = r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\train.csv"

USECOLS = ["date","store_nbr","item_nbr","unit_sales","onpromotion"]
DTYPES  = {"store_nbr":"int16","item_nbr":"int32","unit_sales":"float32"}

df = pd.read_csv(PATH, usecols=USECOLS, parse_dates=["date"], dtype=DTYPES, low_memory=False)

print("Shape:", df.shape)                 # → rows, cols
print(df.head(3))                         # → first rows look sane?
print(df.info())                          # → dtypes are as expected
print("Date range:", df["date"].min(), "→", df["date"].max())

na = df.isna().sum().sort_values(ascending=False)
print("Missing values (top):\n", na.head(10))   # → `onpromotion` will have many NaNs (expected)


Shape: (125497040, 5)
        date  store_nbr  item_nbr  unit_sales onpromotion
0 2013-01-01         25    103665         7.0         NaN
1 2013-01-01         25    105574         1.0         NaN
2 2013-01-01         25    105575         2.0         NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   store_nbr    int16         
 2   item_nbr     int32         
 3   unit_sales   float32       
 4   onpromotion  object        
dtypes: datetime64[ns](1), float32(1), int16(1), int32(1), object(1)
memory usage: 3.0+ GB
None
Date range: 2013-01-01 00:00:00 → 2017-08-15 00:00:00
Missing values (top):
 onpromotion    21657651
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
dtype: int64


### 2) Standardize Schema

In [ ]:

df["onpromotion"] = (
    df["onpromotion"]
      .map({True: True, False: False, "True": True, "False": False, 1: True, 0: False})
      .astype("boolean")   # nullable boolean to avoid FutureWarning
      .fillna(False)       # NA = not on promotion
      .astype(bool)        # final dtype: bool
)

df = df.dropna(subset=["unit_sales"])  

print("onpromotion dtype:", df["onpromotion"].dtype)    # Output: bool
print("onpromotion uniques:", df["onpromotion"].unique())  # Output: [True False]
print("Remaining NA counts:\n", df.isna().sum())       


onpromotion dtype: bool
onpromotion uniques: [False  True]
Remaining NA counts:
 date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
dtype: int64


### 3) Cleaning:

In [ ]:


df["was_return"] = df["unit_sales"] < 0

dup_before = df.duplicated(subset=["date","store_nbr","item_nbr"]).sum()
print("Duplicate keys before groupby:", dup_before)

df = (df.groupby(["date","store_nbr","item_nbr"], as_index=False)
        .agg(unit_sales=("unit_sales","sum"),
             onpromotion=("onpromotion","max"),
             was_return=("was_return","any")))

dup_after = df.duplicated(subset=["date","store_nbr","item_nbr"]).sum()
print("Duplicate keys after groupby:", dup_after)   


Duplicate keys before groupby: 0
Duplicate keys after groupby: 0


### 4) Clipping negatives, capped extreme outliers and applied log1p transformation:

In [ ]:

df["unit_sales_clipped"] = df["unit_sales"].clip(lower=0).astype("float32")

cap = df["unit_sales_clipped"].quantile(0.999) 
df["unit_sales_capped"] = np.minimum(df["unit_sales_clipped"], cap).astype("float32")

df["unit_sales_log1p"] = np.log1p(df["unit_sales_capped"]).astype("float32")

print(df["unit_sales_clipped"].describe(percentiles=[0.99,0.999]))


count    1.254970e+08
mean     8.556009e+00
std      2.352696e+01
min      0.000000e+00
50%      4.000000e+00
99%      7.100000e+01
99.9%    2.127619e+02
max      8.944000e+04
Name: unit_sales_clipped, dtype: float64


### 5) Validation (no null keys, unique keys, clipped ≥ 0):

In [ ]:


print("Nulls in keys:\n", df[["date","store_nbr","item_nbr"]].isna().sum())
print("Duplicate keys:", df.duplicated(subset=["date","store_nbr","item_nbr"]).sum())
print("Min clipped:", df["unit_sales_clipped"].min())

assert df[["date","store_nbr","item_nbr"]].isna().sum().sum() == 0
assert df.duplicated(subset=["date","store_nbr","item_nbr"]).sum() == 0
assert df["unit_sales_clipped"].min() >= 0.0
print("Validation passed")


Nulls in keys:
 date         0
store_nbr    0
item_nbr     0
dtype: int64
Duplicate keys: 0
Min clipped: 0.0
Validation passed


### 6)  Save to Parquet (partitioned):

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

out_dir = Path("data/processed/train_clean_parquet")
out_dir.mkdir(parents=True, exist_ok=True)

df["dow"]   = df["date"].dt.dayofweek.astype("int8")
df["year"]  = df["date"].dt.year.astype("int16")
df["month"] = df["date"].dt.month.astype("int8")

pq.write_to_dataset(
    pa.Table.from_pandas(df, preserve_index=False),
    root_path=str(out_dir),
    partition_cols=["year","month"],
    compression="zstd",
    use_dictionary=True
)
print("Saved →", out_dir)




NameError: name 'df' is not defined

### ITEMS.CSV

In [12]:
import pandas as pd  

#Load items.csv
items = pd.read_csv(r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\items.csv")


print(items.shape)
print(items.head())
print(items.info())

# Check missing values
print(items.isnull().sum())

# Check duplicates
print(items.duplicated().sum())

# Standardize family names (strip spaces, lowercase)
items["family"] = items["family"].str.strip().str.lower()

# Convert perishable to boolean.
items["perishable"] = items["perishable"].astype(bool)

items.to_parquet("../data/processed/items.parquet", index=False)

(4100, 4)
   item_nbr        family  class  perishable
0     96995     GROCERY I   1093           0
1     99197     GROCERY I   1067           0
2    103501      CLEANING   3008           0
3    103520     GROCERY I   1028           0
4    103665  BREAD/BAKERY   2712           1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_nbr    4100 non-null   int64 
 1   family      4100 non-null   object
 2   class       4100 non-null   int64 
 3   perishable  4100 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 128.2+ KB
None
item_nbr      0
family        0
class         0
perishable    0
dtype: int64
0


### STORES.CSV

In [10]:
# Load stores.csv
stores = pd.read_csv(r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\stores.csv")


# Basic checks
print(stores.shape)
print(stores.head())
print(stores.isna().sum())

# Clean text columns, Strip whitespace, standardize casing (city/state = Title, type = uppercase).
stores["city"] = stores["city"].str.strip().str.title()
stores["state"] = stores["state"].str.strip().str.title()
stores["type"] = stores["type"].str.strip().str.upper()

# Dtypes
stores["store_nbr"] = stores["store_nbr"].astype("int16")
stores["cluster"] = stores["cluster"].astype("Int16")

# Check store_nbr uniqueness
assert stores["store_nbr"].is_unique

# Save to parquet
stores.to_parquet("../data/processed/stores.parquet", index=False)


(54, 5)
   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4
store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64


### TRANSACTIONS.CSV

- Convert `date` to a datetime, `store_nbr` to a small int and `transactions` to int.
- Check for duplicates on `store_nbr`and `date`
- Counts should be non-negative.

In [9]:
transactions = pd.read_csv(r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\transactions.csv")

# Inspect
print("Shape:", transactions.shape)
print(transactions.head())
print(transactions.isna().sum())

# Dtypes
transactions["date"] = pd.to_datetime(transactions["date"], errors="coerce")
transactions["store_nbr"] = transactions["store_nbr"].astype("int16")
transactions["transactions"] = pd.to_numeric(transactions["transactions"], errors="coerce").astype("Int32")

# Checks
dupes = transactions.duplicated(subset=["store_nbr", "date"]).sum()
assert dupes == 0, f"Duplicate store/date pairs found: {dupes}"

negatives = (transactions["transactions"] < 0).sum()
assert negatives == 0, f"Negative transaction counts found: {negatives}"

date_min, date_max = transactions["date"].min(), transactions["date"].max()
print(f"Date range: {date_min} → {date_max}")

# Save
transactions.to_parquet("../data/processed/transactions.parquet", index=False)


Shape: (83488, 3)
         date  store_nbr  transactions
0  2013-01-01         25           770
1  2013-01-02          1          2111
2  2013-01-02          2          2358
3  2013-01-02          3          3487
4  2013-01-02          4          1922
date            0
store_nbr       0
transactions    0
dtype: int64
Date range: 2013-01-01 00:00:00 → 2017-08-15 00:00:00


### OIL.CSV (economic indicator)

- Convert `date` to datetime, set as index for easier resampling.
- Interpolate if missing days.

In [8]:
oil = pd.read_csv(r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\oil.csv")

# Convert date
oil["date"] = pd.to_datetime(oil["date"], errors="coerce")

# Basic checks
print("Shape:", oil.shape)
print(oil.isna().sum())
print(oil["date"].min(), "→", oil["date"].max())

# Sort and set index
oil = oil.sort_values("date").set_index("date")

# Rename for clarity
oil = oil.rename(columns={"dcoilwtico": "oil_price"})


# Fill any remaining edges
oil["oil_price"] = oil["oil_price"].ffill().bfill()


print("Remaining NAs:", oil["oil_price"].isna().sum())


# Reset index for saving
oil = oil.reset_index()

# Save
oil.to_parquet("../data/processed/oil.parquet", index=False)

Shape: (1218, 2)
date           0
dcoilwtico    43
dtype: int64
2013-01-01 00:00:00 → 2017-08-31 00:00:00
Remaining NAs: 0


### HOLIDAYS_EVENTS.CSV

In [ ]:
import pandas as pd
from pathlib import Path
holidays_events = pd.read_csv(r"C:\Users\Yizi\New folder\Payday-surge-favorita\data\raw\holidays_events.csv")

holidays_events["date"] = pd.to_datetime(holidays_events["date"], errors="coerce")

# Normalize text
for c in ["type", "locale", "description"]:
    holidays_events[c] = holidays_events[c].astype(str).str.strip()

# Drop originals of transferred holidays
holidays_events = holidays_events[~holidays_events["transferred"]]

# Define flags
holidays_events["is_event"] = holidays_events["type"].str.lower().eq("event")
holidays_events["is_holiday"] = holidays_events["type"].isin(["Holiday", "Additional", "Bridge"])
holidays_events.loc[holidays_events["type"].eq("Work Day"), "is_holiday"] = False  

# Collapse by date 
rank = {"National": 3, "Regional": 2, "Local": 1}
holidays_events["rank"] = holidays_events["locale"].map(rank)
holidays_events = (holidays_events.sort_values(["date", "rank"], ascending=[True, False])
          .groupby("date", as_index=False)
          .agg({"is_holiday": "max", "is_event": "max", "type": "first"}))

holidays_events.to_parquet("../data/processed/holidays.parquet", index=False)

